In [ ]:
!pip install autocorrect

     |████████████████████████████████| 622 kB 5.6 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=4ba592c1a0741b217eb81e2713a16d34a36130603c6074a0bce8ddcc07af5a35
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import codecs
import math
import nltk
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from autocorrect import Speller
import random


In [ ]:
with codecs.open('/content/drive/MyDrive/Deep Learning/parallel/IITB.en-hi.hi', encoding='utf-8') as f:
    data_hindi_train = f.read()
with codecs.open('/content/drive/MyDrive/Deep Learning/parallel/IITB.en-hi.en', encoding='utf-8') as f:
    data_english_train = f.read()

with codecs.open('/content/drive/MyDrive/Deep Learning/dev_test/dev.hi', encoding='utf-8') as f:
    data_hindi_val = f.read()
with codecs.open('/content/drive/MyDrive/Deep Learning/dev_test/dev.en', encoding='utf-8') as f:
    data_english_val = f.read()

with codecs.open('/content/drive/MyDrive/Deep Learning/dev_test/test.hi', encoding='utf-8') as f:
    data_hindi_test = f.read()
with codecs.open('/content/drive/MyDrive/Deep Learning/dev_test/test.en', encoding='utf-8') as f:
    data_english_test = f.read()

FileNotFoundError: ignored

In [ ]:
data_hindi_rows_train=data_hindi_train.split("\n")
data_english_rows_train=data_english_train.split("\n")

data_hindi_rows_val=data_hindi_val.split("\n")
data_english_rows_val=data_english_val.split("\n")

data_hindi_rows_test=data_hindi_test.split("\n")
data_english_rows_test=data_english_test.split("\n")

regex = re.compile('[%s]' % re.escape(string.punctuation))


In [ ]:
train_instances=1609682
data_hindi_rows_train=data_hindi_rows_train[0:train_instances]
data_english_rows_train=data_english_rows_train[0:train_instances]

val_instances=520
data_hindi_rows_val=data_hindi_rows_val[0:val_instances]
data_english_rows_val=data_english_rows_val[0:val_instances]

test_instances=2507
data_hindi_rows_test=data_hindi_rows_test[0:test_instances]
data_english_rows_test=data_english_rows_test[0:test_instances]

In [ ]:
def preprocess_eng(english_row):
    sent=regex.sub(' ',english_row)
    tokens= word_tokenize(sent) 
    #spell = Speller(lang='en')  
    processed_sent=[]
    for token in tokens:
        key=token.lower()   #spell(token.lower())
        processed_sent.append(key)
        
    return ' '.join(processed_sent)

def preprocess_hi(hindi_row):
    sent=regex.sub(' ',hindi_row)
    tokens= word_tokenize(sent) 
    processed_sent=[]
    for token in tokens:
        tok=token
        if ord('।') == ord(token[-1]):
            tok=token[:-1]
        processed_sent.append(tok)  

    return ' '.join(processed_sent)  

In [ ]:
MAX_LENGTH=30
EOS_token=1
SOS_token=0

ID2Vec_eng={}
word2ID_eng={"SOS": 0, "EOS": 1, "UNK": 2}
ID2word_eng={0: "SOS", 1: "EOS", 2: "UNK"}

ID2Vec_hi={}
word2ID_hi={"SOS": 0, "EOS": 1, "UNK": 2}
ID2word_hi={0: "SOS", 1: "EOS", 2: "UNK"}
combo_train = []
combo_val = []
combo_test = []

def normalizeAndBuildHindiVocab(data_hindi_rows, combo):
    print(len(data_hindi_rows))
    id=len(word2ID_hi)
    for hindi_row in data_hindi_rows:

        sent=preprocess_hi(hindi_row)
        combo.append([sent])
        for token in sent.split(' '):
            if token not in word2ID_hi:
                word2ID_hi[token]=id
                ID2word_hi[id]=token
                ID2Vec_hi[id]=[]
                id=id+1  
    return combo      
            

def normalizeAndBuildEnglishVocab(data_english_rows, combo):
    print(len(data_english_rows))
    count=0
    id=len(word2ID_eng)
    for english_row in data_english_rows:       
        sent=preprocess_eng(english_row)
        combo[count].append(sent)
        for token in sent.split(' '):
            if token not in word2ID_eng:
                word2ID_eng[token]=id
                ID2word_eng[id]=token
                ID2Vec_eng[id]=[]
                id=id+1
        count=count+1
    return combo
            

In [ ]:
normalizeAndBuildHindiVocab(data_hindi_rows_train, combo_train )
normalizeAndBuildEnglishVocab(data_english_rows_train, combo_train)

normalizeAndBuildHindiVocab(data_hindi_rows_val, combo_val)
normalizeAndBuildEnglishVocab(data_english_rows_val, combo_val)

normalizeAndBuildHindiVocab(data_hindi_rows_test, combo_test)
normalizeAndBuildEnglishVocab(data_english_rows_test, combo_test)

1609682
1609682
520
520
2507
2507


[['आपकी कार में ब्लैक बॉक्स', 'a black box in your car'],
 ['जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं वहीं बहुत से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है',
  'as america s road planners struggle to find the cash to mend a crumbling highway system many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car'],
 ['यह डिवाइस जो मोटर चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है आजकल अमेरिका की प्रमुख सड़कों का वित्त पोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के लिए वाशिंगटन और राज्य नियोजन कार्यालय के लिए एक विवादास्पद प्रयास का मुद्दा बन चुका है',
  'the devices which track every mile a motorist drives and transmit that information to bureaucrats are at the center of a controversial attempt in washington and state planning offices to overhaul the outda

In [ ]:
import pickle
with open('word2ID_eng.pkl', 'wb') as f:
    pickle.dump(word2ID_eng, f)
        
with open('ID2word_eng.pkl', 'wb') as f:
    pickle.dump(ID2word_eng, f)
    
with open('word2ID_hi.pkl', 'wb') as f:
    pickle.dump(word2ID_hi, f)

with open('ID2word_hi.pkl', 'wb') as f:
    pickle.dump(ID2word_hi, f)

In [ ]:
with open('word2ID_eng.pkl', 'rb') as f:
        word2ID_eng = pickle.load(f)
with open('ID2word_eng.pkl', 'rb') as f:
        ID2word_eng = pickle.load(f)
with open('word2ID_hi.pkl', 'rb') as f:
        word2ID_hi = pickle.load(f)
with open('ID2word_hi.pkl', 'rb') as f:
        ID2word_hi = pickle.load(f)

In [ ]:
with open('combo_train.pkl', 'wb') as f:
  pickle.dump(combo_train, f)
with open('combo_val.pkl', 'wb') as f:
  pickle.dump(combo_val, f)
with open('combo_test.pkl', 'wb') as f:
  pickle.dump(combo_test, f)

In [ ]:
with open('combo_train.pkl', 'rb') as f:
        combo_train= pickle.load(f)
with open('combo_val.pkl', 'rb') as f:
        combo_val= pickle.load(f)
with open('combo_test.pkl', 'rb') as f:
        combo_test= pickle.load(f)

In [ ]:
#using glove embeddings for english words
count=0
cnt=0
with open('/content/drive/MyDrive/Deep Learning/glove.6B.50d.txt/glove.6B.50d.txt','r',encoding="utf8") as file: 
  
    for index, line in enumerate(file): 
        values = line.split()
        word=values[0]
        if word in word2ID_eng.keys():
            cnt=cnt+1
            ID2Vec_eng[word2ID_eng[word]]= np.asarray(values[1:], dtype=np.float32)     
        count=count+1

In [ ]:
#english words not present in the glove embeddings
mispell=[]
cnt=0
for i in word2ID_eng.keys():
    try:
        a=ID2Vec_eng[word2ID_eng[i]].shape
    except:
        print(i)
        mispell.append(i)
        cnt=cnt+1
print(cnt)

SOS
EOS
UNK
accerciser
ipython
acti
plication
lection
mponent
ktop
ypertext
reamable
accessibles
dogtail
ldtp
alidate
focusable
rolename
num1
num2
logout
hotkeys
©
pluginview
pluginviews
aisleriot
merchantability
easthaven
pileon
spiderette
02d
game…
tatistics
themes…
statusbar
redeals
redeal
02110
launchable
anjuta
execvp

pixbuf
pixmap
scratchbox
onfigure
autogenerate
tarball
mpile
complie
autotools
gobject
autogen
dialog1
dialog2
disp
sysm
pect
autoformat
parenth
autosave
mment
uncomment
histor
bookmar
formating
calltip
whitespaces
mesg
makefiles
dialog3
gtodo
unfiled
changelog
csharp
starget
ject
confim
infromation
sampleplugin
eplace
vious
repositry
globale
claration
valgrind
ustomize
splashscreen
automake
ompile
makefile
tooltips
dialog4
cachegrind
helgrind
memcheck
unversioned
pixbufs
gtkmm
gcj
addrcheck
widthxheight
xoff
yoff
autocompletion
calltips
gtksourceview
devhelp
gtype
paramspec
gbfproject
symboldb
parenthese
project…
browse…
breakpoint…
running…
as…
print…
number…
open

In [ ]:
with open('mispell.pkl', 'wb') as f:
  pickle.dump(mispell, f)

In [ ]:
with open('mispell.pkl', 'rb') as f:
        mispell= pickle.load(f)

In [ ]:
 def sentenceToIDs_eng(sentence):
    list=[]
    for token in sentence.split(' '):
        if token in mispell:
            list.append(word2ID_eng['a'])
        elif token not in word2ID_eng.keys():
          list.append(word2ID_eng['UNK'])
        else:
            list.append(word2ID_eng[token])
    list.append(word2ID_eng['EOS'])  
    return list

def sentenceToIDs_hi(sentence):
    list= []
    for token in sentence.split(' '):
      if token not in word2ID_hi.keys():
          list.append(word2ID_hi['UNK'])
      else:
        list.append(word2ID_hi[token])
    list.append(word2ID_hi['EOS'])  
    return list

In [ ]:
# engs = []
# hins = []

for id in range(100000, 500000): 
  hins.append(sentenceToIDs_hi(combo_train[id][0]))
  engs.append(sentenceToIDs_eng(combo_train[id][1]))
    
eng_data=np.array(engs)
hin_data=np.array(hins)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [ ]:
with open('eng_data.pkl', 'wb') as f:
  pickle.dump(eng_data, f)
with open('hin_data.pkl', 'wb') as f:
  pickle.dump(hin_data, f)

In [ ]:
with open('eng_data.pkl', 'rb') as f:
        eng_data= pickle.load(f)
with open('hin_data.pkl', 'rb') as f:
        hin_data= pickle.load(f)

In [ ]:
with open('eng_data.pkl', 'wb') as f:
    pickle.dump(eng_data, f)
        
with open('hin_data.pkl', 'wb') as f:
    pickle.dump(hin_data, f)
    